In [1]:
#!pip install 'monai[all]'

In [2]:
#Thank god-> Starting from Python 3.7, dictionaries maintain the insertion order.

import importlib
import subprocess

def check_and_install_packages(package_list):
    missing_packages = []

    for package in package_list:
        try:
            __import__(package)
        except ImportError:
            missing_packages.append(package)

    if missing_packages:
        print("Installing missing packages:")
        for package in missing_packages:
            subprocess.run(["pip", "install", package])
            print(f"Installed {package}")
    else:
        print("All required packages are already installed.")

# List of packages to check and install if necessary
required_packages = [
    "numpy",
    "matplotlib",
    "pandas",
    #"cv2",
    "imageio",
    "PIL",
    "torch",
    "monai",
    "sys"
]

check_and_install_packages(required_packages)

In [3]:
import sys
print(sys.version)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import pandas as pd
import os
import cv2 as cv
import imageio
from PIL import Image
import imageio.v3 as iio

In [6]:
#updated_raw_dict, new_analyzed_dict
import PIL

In [7]:
def convert_to_numpy(data):
    if isinstance(data, np.ndarray):
        return data.astype(np.float64)
    elif isinstance(data, PIL.Image.Image):
        return np.array(data).astype(np.float64)
    elif isinstance(data, torch.Tensor):
        return data.numpy().astype(np.float64)
    # Add more conversion cases if needed
    else:
        raise ValueError("Unsupported data format")

def check_and_convert_format(updated_raw_dict, new_analyzed_dict):
    converted_raw_dict = {key: convert_to_numpy(value) for key, value in updated_raw_dict.items()}
    converted_analyzed_dict = {key: convert_to_numpy(value) for key, value in new_analyzed_dict.items()}

    return converted_raw_dict, converted_analyzed_dict


In [8]:
import pickle



In [9]:
import torch
torch.cuda.empty_cache()
#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'MAX_SPLIT_SIZE_MB=256'  # Adjust the size as needed

In [10]:
#weight_values = [1.0, 1.5, 2.0, 2.5, 5.0, 10.0, 20.0]  # Example weight values to experiment with

In [11]:
#!pip install tensorboardX
import tensorboardX

In [12]:
import random
# Set a random seed for Python's built-in random module
random_seed = 42
random.seed(random_seed)

# Set a random seed for NumPy
np.random.seed(random_seed)

# Set a random seed for PyTorch on both CPU and CUDA (if available)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

In [13]:
#!pip install tensorboard

In [14]:
import logging
import os
import sys
import tempfile
from glob import glob

In [15]:
import torch
from torch.cuda.amp import autocast, GradScaler
from torch.utils.tensorboard import SummaryWriter
import torch.nn.utils as nn_utils
import torch.autograd as autograd
from torch.optim.lr_scheduler import StepLR

In [16]:
import psutil

In [17]:
import monai

In [18]:
from monai.data import create_test_image_2d, list_data_collate, decollate_batch, DataLoader

In [19]:
from monai.inferers import sliding_window_inference


In [20]:
from monai.metrics import DiceMetric

In [21]:
from monai.transforms import (
    Activations,
    EnsureChannelFirstd,
    Compose,
    ScaleIntensityd,
    ToTensor,
    AsDiscrete
)
#   DivisiblePadd deleted

In [22]:
from monai.transforms import Transform
#from monai.visualize import plot_2d_or_3d_image

In [23]:
from monai.networks.layers import Norm
from monai.transforms import Resized

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [25]:
#!pip install scikit-learn

In [26]:
from sklearn.model_selection import train_test_split
import random


In [27]:
from PIL import Image

In [28]:


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_files, segmentation_files, transform=None):
        self.input_files = input_files
        self.segmentation_files = segmentation_files
        self.transform = transform

    def __len__(self):
        return len(self.input_files)

    def __getitem__(self, idx):
        input_path = self.input_files[idx]
        segmentation_path = self.segmentation_files[idx]

        # Load input image (TIFF)
        input_image = Image.open(input_path)
        input_image = np.array(input_image, dtype=np.float32)  # Convert to NumPy array

        # Load segmentation image (TIFF)
        segmentation_image = Image.open(segmentation_path)
        segmentation_image = np.array(segmentation_image, dtype=np.float32)  # Convert to NumPy array

        # Initialize variables
        input_data = input_image
        segmentation_data = segmentation_image

        if self.transform is not None:
            # Apply the specified transforms to input_data and segmentation_data
            transformed_data = self.transform({"img": input_data, "seg": segmentation_data})
            input_data, segmentation_data = transformed_data["img"], transformed_data["seg"]

        return {"img": input_data, "seg": segmentation_data}






max_grad_norm = 1.0
# Define a custom transform to threshold the segmentation tensor
class ThresholdSegmentation(Transform):
    def __init__(self, keys, threshold=0):
        super().__init__()
        self.keys = keys
        self.threshold = threshold

    def __call__(self, data):
        for key in self.keys:
            if key == "seg":
                data[key] = (data[key] > self.threshold).float()
        return data

import matplotlib.pyplot as plt
from monai.transforms import ToNumpy




def main():
    log_dir = "D:/Aytekin/CovidMasterThesis/runs/resize_512_green_alldata_bigmodel_lr_1e3_epoch20_64_1024"
    monai.config.print_config()
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)


    #print(len(updated_raw_dict))
    #print(len(new_analyzed_dict))
    # Check and convert data format only once
    #converted_raw_dict, converted_analyzed_dict = check_and_convert_format(updated_raw_dict, new_analyzed_dict)

    # Remove red for now!!!!
    #converted_raw_dict = {k: v for k, v in converted_raw_dict.items() if 'red' not in k}

    # Remove elements where the key contains "red" for converted_analyzed_dict
    #converted_analyzed_dict = {k: v for k, v in converted_analyzed_dict.items() if 'red' not in k}


    # Check if the dictionaries contain the same length or not, then create train val test:
    #if len(converted_raw_dict) != len(converted_analyzed_dict):
    #    raise ValueError("The lengths of converted_raw_dict and converted_analyzed_dict do not match.")

    #num_images = len(converted_raw_dict)
    #print("Length of converted_raw_dict to be used: {}".format(num_images))
    # Calculate the number of images for training, validation, and test, e.g., using an 80-10-10 split

    #
    #converted_raw_dict.keys()
    
    
    

        # define transforms for image and segmentation
    train_transforms = Compose(
        [
            #ToTensor(),
            EnsureChannelFirstd(keys=["img", "seg"], channel_dim="no_channel"),  # Use channel_dim=-1 for NumPy arrays
            ScaleIntensityd(keys=["img", "seg"]),
            Resized(keys=["img", "seg"],spatial_size=(512, 512), mode="nearest"),
            ThresholdSegmentation(keys=["img", "seg"], threshold=0),

            #DivisiblePadd(keys=["img", "seg"],k=16),

        ]
    )
    val_transforms = Compose(
        [
            #ToTensor(),
            EnsureChannelFirstd(keys=["img", "seg"], channel_dim="no_channel"),  # Use channel_dim=-1 for NumPy arrays
            ScaleIntensityd(keys=["img", "seg"]),
            Resized(keys=["img", "seg"],spatial_size=(512, 512), mode="nearest"),
            ThresholdSegmentation(keys=["img", "seg"], threshold=0),
            #DivisiblePadd(keys=["img", "seg"],k=16),


        ]
    )
    
    
    
    


    # Directory paths for input and segmentation images
    input_image_dir = r"E:\Aytekin\Data\Input"
    segmentation_dir = r"E:\Aytekin\Data\Segmentation"
    test_input_dir = r"E:\Aytekin\Data\Test\Input"

    # List of strings to exclude from image names
    exclude_strings = ["12_0500", "29_0520", "01_0520", "08_0420", "29_0820","29_1120","29_1420","29_1720","29_2020","29_2320"]  # Add more strings if needed

    # List all .tif image files in the directories and filter out images with excluded strings
    input_image_files = [os.path.join(input_image_dir, filename) for filename in os.listdir(input_image_dir) if filename.endswith(".tif") and all(exclude not in filename for exclude in exclude_strings)]
    segmentation_files = [os.path.join(segmentation_dir, filename) for filename in os.listdir(segmentation_dir) if filename.endswith(".tif") and all(exclude not in filename for exclude in exclude_strings)]

    # Get the list of files in the "Test Input" folder
    test_input_files = [os.path.splitext(os.path.basename(filename))[0] for filename in os.listdir(test_input_dir) if filename.endswith(".tif")]

    # Exclude images with the same names as those in the "Test Input" folder
    input_image_files = [filename for filename in input_image_files if os.path.splitext(os.path.basename(filename))[0] not in test_input_files]
    segmentation_files = [filename for filename in segmentation_files if os.path.splitext(os.path.basename(filename))[0] not in test_input_files]

    # List all .tif image files in the directories
    input_image_files = [os.path.join(input_image_dir, filename) for filename in os.listdir(input_image_dir) if filename.endswith(".tif")]
    segmentation_files = [os.path.join(segmentation_dir, filename) for filename in os.listdir(segmentation_dir) if filename.endswith(".tif")]

    # Extract file names without extensions
    input_image_names = [os.path.splitext(os.path.basename(filename))[0] for filename in input_image_files]
    segmentation_names = [os.path.splitext(os.path.basename(filename))[0] for filename in segmentation_files]

    # Find the common file names between input and segmentation
    common_names = set(input_image_names) & set(segmentation_names)

    # Filter input_image_files and segmentation_files based on common names
    input_image_files = [filename for filename in input_image_files if os.path.splitext(os.path.basename(filename))[0] in common_names]
    segmentation_files = [filename for filename in segmentation_files if os.path.splitext(os.path.basename(filename))[0] in common_names]




    # Split the data into training and validation sets
    input_image_train, input_image_val, segmentation_train, segmentation_val = train_test_split(
        input_image_files, segmentation_files, test_size=0.2, random_state=42)

    # Create instances of the custom dataset for training and validation
    train_dataset = CustomDataset(input_image_train, segmentation_train, transform=train_transforms)
    val_dataset = CustomDataset(input_image_val, segmentation_val, transform=val_transforms)

    # Define data loaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=0, collate_fn=list_data_collate, pin_memory=torch.cuda.is_available())
    val_loader = DataLoader(val_dataset, batch_size=1, num_workers=0, collate_fn=list_data_collate)








    # define dataset, data loader
    #check_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
    # use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
    #check_loader = DataLoader(check_ds, batch_size=2, num_workers=0, collate_fn=list_data_collate)
    #check_data = monai.utils.misc.first(check_loader)
    #print(check_data["img"].shape, check_data["seg"].shape)
    #print("------\n")
    #print(torch.unique(check_data["seg"]))
    # create a training data loader
    #train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
    # use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
    #train_loader = DataLoader(
    #    train_ds,
    #    batch_size=2,
    #    shuffle=False,
    #    num_workers=0,
    #    collate_fn=list_data_collate,
    #    pin_memory=torch.cuda.is_available(),
    #)
    # create a validation data loader
    #val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
    #val_loader = DataLoader(val_ds, batch_size=1, num_workers=0, collate_fn=list_data_collate)
    dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
    post_trans = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
    # create UNet, DiceLoss and Adam optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    ######## Different Models Can be Tested ##############
    model = monai.networks.nets.UNet(
        spatial_dims=2,
        in_channels=1,
        out_channels=1,
        channels=(64, 128, 256, 512, 1024),
        strides=(2, 2, 2, 2),
        num_res_units=3,
        norm = Norm.BATCH,
    ).to(device)
    
    
    ######## Different Models Can be Tested ##############
    #model = monai.networks.nets.UNet(
    #    spatial_dims=2,
    #    in_channels=1,
    #    out_channels=1,
    #    channels=(64, 128, 256, 512, 1024, 2048),
    #    strides=(2, 2, 2, 2, 2),
    #    num_res_units=4,
    #    norm=None  # Remove normalization layers
    #).to(device)
    
    loss_function = monai.losses.DiceLoss(include_background=True,sigmoid=True,smooth_nr=1e-05, smooth_dr=1e-05) #, weight = [1/0.9892240500840984, 1/0.01077594991590156]
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3 ) #1e-3
    #optimizer = Novograd(model.parameters(), lr=1e-2, clip=1.0)
    # start a typical PyTorch training
    val_interval = 2
    best_metric = -1
    best_metric_epoch = -1
    epoch_loss_values = list()
    metric_values = list()
    writer = SummaryWriter(log_dir=log_dir)


    autograd.set_detect_anomaly(True)
    scaler = GradScaler()

    # Define the step size and gamma (decay factor)
    step_size = 1200  # Adjust this according to your training schedule
    gamma = 0.5     # Adjust this according to your preference

    # Create a learning rate scheduler
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    # Create a DiceMetric instance for training
    train_dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)

    for epoch in range(20):
        print("-" * 20)
        print(f"epoch {epoch + 1}/{20}")
        model.train()
        epoch_loss = 0
        step = 0
        # Clear the DiceMetric for the new epoch
        train_dice_metric.reset()
        for batch_data in train_loader:
            step += 1
            inputs, labels = batch_data["img"].to(device), batch_data["seg"].to(device)
            optimizer.zero_grad()
            with autocast():
                outputs = model(inputs)
                loss = loss_function(outputs, labels)

            # Calculate the Dice score during training
            post_outputs = post_trans(outputs)
            train_dice_metric(y_pred=post_outputs, y=labels)

            scaler.scale(loss).backward()
            #scaler.unscale_(optimizer)  # Unscales the gradients of optimizer's assigned params in-place
            ############## Gradient Clipping Part ###################
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Add this line to clip gradients
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()
            epoch_len = len(train_dataset) // train_loader.batch_size
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)




        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        # Calculate the average Dice score for the epoch
        train_dice_score = train_dice_metric.aggregate().item()
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}, train_dice: {train_dice_score:.4f}")
        writer.add_scalar("train_dice", train_dice_score, epoch + 1)


        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
        scheduler.step()
        if (epoch + 1) % val_interval == 0:
            model.eval()
            with torch.no_grad():
                val_images = None
                val_labels = None
                val_outputs = None
                for val_data in val_loader:
                    val_images, val_labels = val_data["img"].to(device), val_data["seg"].to(device)
                    roi_size = (96, 96)
                    sw_batch_size = 4
                    val_outputs = sliding_window_inference(val_images, roi_size, sw_batch_size, model)
                    val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]
                    # compute metric for current iteration
                    dice_metric(y_pred=val_outputs, y=val_labels)

                    for i in range(len(val_images)):
                        input_image = val_images[i:i + 1]  # Get the i-th image
                        ground_truth = val_labels[i:i + 1]  # Get the i-th ground truth
                        prediction = val_outputs[i:i + 1]  # Get the i-th prediction

                        # Perform intensity scaling if needed
                        #a_min = 0.0
                        #a_max = 255.0
                        #b_min = 0.0
                        #b_max = 1.0

                        if isinstance(prediction, list):
                            prediction = prediction[0]
                        
                        
                        #input_image_np = input_image.detach().cpu().numpy()  # Convert tensor to NumPy array
                        #ground_truth_np = ground_truth.detach().cpu().numpy()  # Convert tensor to NumPy array
                        #prediction_np = prediction.detach().cpu().numpy()  # Convert tensor to NumPy array
                        
                        #input_image_np = (input_image_np - input_image_np.min()) * (255.0 / (input_image_np.max() - input_image_np.min()))
                        #ground_truth_np = (ground_truth_np - ground_truth_np.min()) * (255.0 / (ground_truth_np.max() - ground_truth_np.min()))
                        #prediction_np = (prediction_np - prediction_np.min()) * (255.0 / (prediction_np.max() - prediction_np.min()))
                        
                        
                        writer.add_image('Validation/Input', input_image, global_step=epoch,dataformats='NCHW')
                        writer.add_image('Validation/GroundTruth', ground_truth, global_step=epoch,dataformats='NCHW')
                        writer.add_image('Validation/Prediction', prediction, global_step=epoch,dataformats='CHW')


                # aggregate the final mean dice result
                metric = dice_metric.aggregate().item()
                # reset the status for next validation round
                dice_metric.reset()
                metric_values.append(metric)
                if metric > best_metric:
                    best_metric = metric
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(), "512green_alldata_seg2d_epoch20_64_1024_higherlr.pth")
                    print("saved new best metric model")
                print(
                    "current epoch: {} current mean dice: {:.4f} best mean dice: {:.4f} at epoch {}".format(
                        epoch + 1, metric, best_metric, best_metric_epoch
                    )
                )
                writer.add_scalar("val_mean_dice", metric, epoch + 1)
                # plot the last model output as GIF image in TensorBoard with the corresponding image and label




    autograd.set_detect_anomaly(False)
    writer.close()






if __name__ == "__main__":
    main()

MONAI version: 1.2.0
Numpy version: 1.26.0
Pytorch version: 2.0.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: C:\Users\ge59gor\AppData\Local\anaconda3\envs\deepsight\Lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: NOT INSTALLED or UNKNOWN VERSION.
scikit-image version: 0.22.0
Pillow version: 9.4.0
Tensorboard version: 2.14.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.2
tqdm version: NOT INSTALLED or UNKNOWN VERSION.
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 2.0.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details ab

247/1377, train_loss: 0.8421
248/1377, train_loss: 0.8564
249/1377, train_loss: 0.8326
250/1377, train_loss: 0.8418
251/1377, train_loss: 0.8279
252/1377, train_loss: 0.8527
253/1377, train_loss: 0.7635
254/1377, train_loss: 0.8002
255/1377, train_loss: 0.8001
256/1377, train_loss: 0.7916
257/1377, train_loss: 0.8103
258/1377, train_loss: 0.8328
259/1377, train_loss: 0.7974
260/1377, train_loss: 0.7914
261/1377, train_loss: 0.7696
262/1377, train_loss: 0.8087
263/1377, train_loss: 0.7671
264/1377, train_loss: 0.7636
265/1377, train_loss: 0.8264
266/1377, train_loss: 0.8201
267/1377, train_loss: 0.7868
268/1377, train_loss: 0.7168
269/1377, train_loss: 0.7533
270/1377, train_loss: 0.7382
271/1377, train_loss: 0.7822
272/1377, train_loss: 0.7838
273/1377, train_loss: 0.7360
274/1377, train_loss: 0.7937
275/1377, train_loss: 0.7595
276/1377, train_loss: 0.7357
277/1377, train_loss: 0.7673
278/1377, train_loss: 0.7258
279/1377, train_loss: 0.7192
280/1377, train_loss: 0.7155
281/1377, trai

530/1377, train_loss: 0.3862
531/1377, train_loss: 0.3595
532/1377, train_loss: 0.4124
533/1377, train_loss: 0.5264
534/1377, train_loss: 0.4727
535/1377, train_loss: 0.4786
536/1377, train_loss: 0.3704
537/1377, train_loss: 0.3329
538/1377, train_loss: 0.3830
539/1377, train_loss: 0.4184
540/1377, train_loss: 0.3885
541/1377, train_loss: 0.3215
542/1377, train_loss: 0.4339
543/1377, train_loss: 0.2999
544/1377, train_loss: 0.4484
545/1377, train_loss: 0.3298
546/1377, train_loss: 0.3104
547/1377, train_loss: 0.3078
548/1377, train_loss: 0.2858
549/1377, train_loss: 0.3627
550/1377, train_loss: 0.3003
551/1377, train_loss: 0.4360
552/1377, train_loss: 0.3739
553/1377, train_loss: 0.3209
554/1377, train_loss: 0.3863
555/1377, train_loss: 0.3086
556/1377, train_loss: 0.2949
557/1377, train_loss: 0.3552
558/1377, train_loss: 0.3720
559/1377, train_loss: 0.3363
560/1377, train_loss: 0.3536
561/1377, train_loss: 0.3181
562/1377, train_loss: 0.3866
563/1377, train_loss: 0.5076
564/1377, trai

813/1377, train_loss: 0.1683
814/1377, train_loss: 0.1972
815/1377, train_loss: 0.2055
816/1377, train_loss: 0.2059
817/1377, train_loss: 0.2005
818/1377, train_loss: 0.1853
819/1377, train_loss: 0.1951
820/1377, train_loss: 0.2174
821/1377, train_loss: 0.2354
822/1377, train_loss: 0.1906
823/1377, train_loss: 0.2106
824/1377, train_loss: 0.1823
825/1377, train_loss: 0.2267
826/1377, train_loss: 0.1899
827/1377, train_loss: 0.1616
828/1377, train_loss: 0.1987
829/1377, train_loss: 0.1765
830/1377, train_loss: 0.2136
831/1377, train_loss: 0.2068
832/1377, train_loss: 0.2143
833/1377, train_loss: 0.2696
834/1377, train_loss: 0.2035
835/1377, train_loss: 0.2206
836/1377, train_loss: 0.1534
837/1377, train_loss: 0.2295
838/1377, train_loss: 0.1782
839/1377, train_loss: 0.1936
840/1377, train_loss: 0.1784
841/1377, train_loss: 0.1933
842/1377, train_loss: 0.2045
843/1377, train_loss: 0.1628
844/1377, train_loss: 0.2005
845/1377, train_loss: 0.1652
846/1377, train_loss: 0.2350
847/1377, trai

1093/1377, train_loss: 0.1699
1094/1377, train_loss: 0.1892
1095/1377, train_loss: 0.2021
1096/1377, train_loss: 0.1841
1097/1377, train_loss: 0.1770
1098/1377, train_loss: 0.1846
1099/1377, train_loss: 0.1765
1100/1377, train_loss: 0.1974
1101/1377, train_loss: 0.2005
1102/1377, train_loss: 0.1632
1103/1377, train_loss: 0.2277
1104/1377, train_loss: 0.1698
1105/1377, train_loss: 0.1663
1106/1377, train_loss: 0.1928
1107/1377, train_loss: 0.1480
1108/1377, train_loss: 0.2124
1109/1377, train_loss: 0.2043
1110/1377, train_loss: 0.1957
1111/1377, train_loss: 0.1689
1112/1377, train_loss: 0.1686
1113/1377, train_loss: 0.1523
1114/1377, train_loss: 0.2445
1115/1377, train_loss: 0.1742
1116/1377, train_loss: 0.2059
1117/1377, train_loss: 0.2000
1118/1377, train_loss: 0.1904
1119/1377, train_loss: 0.2360
1120/1377, train_loss: 0.2266
1121/1377, train_loss: 0.1834
1122/1377, train_loss: 0.1858
1123/1377, train_loss: 0.1948
1124/1377, train_loss: 0.1836
1125/1377, train_loss: 0.1719
1126/1377,

...

108/1377, train_loss: 0.1197
109/1377, train_loss: 0.1335
110/1377, train_loss: 0.1472
111/1377, train_loss: 0.1280
112/1377, train_loss: 0.1576
113/1377, train_loss: 0.1451
114/1377, train_loss: 0.1541
115/1377, train_loss: 0.1153
116/1377, train_loss: 0.0987
117/1377, train_loss: 0.1446
118/1377, train_loss: 0.1393
119/1377, train_loss: 0.1288
120/1377, train_loss: 0.1541
121/1377, train_loss: 0.1604
122/1377, train_loss: 0.1570
123/1377, train_loss: 0.1916
124/1377, train_loss: 0.1310
125/1377, train_loss: 0.1588
126/1377, train_loss: 0.1459
127/1377, train_loss: 0.1529
128/1377, train_loss: 0.1889
129/1377, train_loss: 0.1565
130/1377, train_loss: 0.1628
131/1377, train_loss: 0.1424
132/1377, train_loss: 0.2041
133/1377, train_loss: 0.1626
134/1377, train_loss: 0.1352
135/1377, train_loss: 0.1628
136/1377, train_loss: 0.1268
137/1377, train_loss: 0.1159
138/1377, train_loss: 0.1981
139/1377, train_loss: 0.1818
140/1377, train_loss: 0.1215
141/1377, train_loss: 0.1201
142/1377, trai

391/1377, train_loss: 0.1525
392/1377, train_loss: 0.1423
393/1377, train_loss: 0.1512
394/1377, train_loss: 0.1565
395/1377, train_loss: 0.1486
396/1377, train_loss: 0.1609
397/1377, train_loss: 0.1763
398/1377, train_loss: 0.1427
399/1377, train_loss: 0.1562
400/1377, train_loss: 0.1142
401/1377, train_loss: 0.1578
402/1377, train_loss: 0.1216
403/1377, train_loss: 0.1485
404/1377, train_loss: 0.1664
405/1377, train_loss: 0.1423
406/1377, train_loss: 0.1234
407/1377, train_loss: 0.1725
408/1377, train_loss: 0.1392
409/1377, train_loss: 0.1711
410/1377, train_loss: 0.1942
411/1377, train_loss: 0.1592
412/1377, train_loss: 0.1285
413/1377, train_loss: 0.1607
414/1377, train_loss: 0.1657
415/1377, train_loss: 0.1663
416/1377, train_loss: 0.1351
417/1377, train_loss: 0.1685
418/1377, train_loss: 0.1150
419/1377, train_loss: 0.1713
420/1377, train_loss: 0.1515
421/1377, train_loss: 0.1671
422/1377, train_loss: 0.1439
423/1377, train_loss: 0.1750
424/1377, train_loss: 0.1572
425/1377, trai

674/1377, train_loss: 0.1543
675/1377, train_loss: 0.1834
676/1377, train_loss: 0.1476
677/1377, train_loss: 0.1293
678/1377, train_loss: 0.1530
679/1377, train_loss: 0.1884
680/1377, train_loss: 0.1586
681/1377, train_loss: 0.1449
682/1377, train_loss: 0.1350
683/1377, train_loss: 0.1088
684/1377, train_loss: 0.2165
685/1377, train_loss: 0.1797
686/1377, train_loss: 0.1448
687/1377, train_loss: 0.1461
688/1377, train_loss: 0.1399
689/1377, train_loss: 0.1657
690/1377, train_loss: 0.1331
691/1377, train_loss: 0.1366
692/1377, train_loss: 0.1703
693/1377, train_loss: 0.1419
694/1377, train_loss: 0.1422
695/1377, train_loss: 0.1651
696/1377, train_loss: 0.1611
697/1377, train_loss: 0.1636
698/1377, train_loss: 0.1213
699/1377, train_loss: 0.2021
700/1377, train_loss: 0.1513
701/1377, train_loss: 0.1095
702/1377, train_loss: 0.1960
703/1377, train_loss: 0.1676
704/1377, train_loss: 0.1449
705/1377, train_loss: 0.1620
706/1377, train_loss: 0.1670
707/1377, train_loss: 0.1526
708/1377, trai

957/1377, train_loss: 0.1737
958/1377, train_loss: 0.1680
959/1377, train_loss: 0.1354
960/1377, train_loss: 0.1665
961/1377, train_loss: 0.1688
962/1377, train_loss: 0.1716
963/1377, train_loss: 0.0987
964/1377, train_loss: 0.1441
965/1377, train_loss: 0.1421
966/1377, train_loss: 0.1452
967/1377, train_loss: 0.1386
968/1377, train_loss: 0.1366
969/1377, train_loss: 0.1185
970/1377, train_loss: 0.1226
971/1377, train_loss: 0.1946
972/1377, train_loss: 0.1191
973/1377, train_loss: 0.1929
974/1377, train_loss: 0.2100
975/1377, train_loss: 0.1182
976/1377, train_loss: 0.1564
977/1377, train_loss: 0.1811
978/1377, train_loss: 0.1434
979/1377, train_loss: 0.1416
980/1377, train_loss: 0.1542
981/1377, train_loss: 0.1380
982/1377, train_loss: 0.1483
983/1377, train_loss: 0.1362
984/1377, train_loss: 0.1428
985/1377, train_loss: 0.1213
986/1377, train_loss: 0.1148
987/1377, train_loss: 0.1793
988/1377, train_loss: 0.1783
989/1377, train_loss: 0.1331
990/1377, train_loss: 0.1468
991/1377, trai

1232/1377, train_loss: 0.1318
1233/1377, train_loss: 0.1598
1234/1377, train_loss: 0.1360
1235/1377, train_loss: 0.1279
1236/1377, train_loss: 0.1739
1237/1377, train_loss: 0.1194
1238/1377, train_loss: 0.1506
1239/1377, train_loss: 0.1336
1240/1377, train_loss: 0.1329
1241/1377, train_loss: 0.1465
1242/1377, train_loss: 0.1751
1243/1377, train_loss: 0.1573
1244/1377, train_loss: 0.1585
1245/1377, train_loss: 0.1555
1246/1377, train_loss: 0.1278
1247/1377, train_loss: 0.1275
1248/1377, train_loss: 0.1532
1249/1377, train_loss: 0.1764
1250/1377, train_loss: 0.1672
1251/1377, train_loss: 0.1291
1252/1377, train_loss: 0.1393
1253/1377, train_loss: 0.1370
1254/1377, train_loss: 0.1440
1255/1377, train_loss: 0.1704
1256/1377, train_loss: 0.1743
1257/1377, train_loss: 0.1308
1258/1377, train_loss: 0.1324
1259/1377, train_loss: 0.1477
1260/1377, train_loss: 0.1590
1261/1377, train_loss: 0.1480
1262/1377, train_loss: 0.1978
1263/1377, train_loss: 0.1397
1264/1377, train_loss: 0.1729
1265/1377,

In [29]:
!tensorboard --logdir=D:/Aytekin/CovidMasterThesis/runs/resize_512_green_alldata_bigmodel_lr_1e3_epoch20_64_1024 --port=6012

^C


In [30]:
print(a)

NameError: name 'a' is not defined

In [ ]:
# import os
#os.system("pkill -f 'tensorboard'")

In [ ]:
#%load_ext tensorboard

In [ ]:
#!pip install --upgrade tensorboard